# 学习率衰减
对于基于一阶梯度进行优化的方法而言，开始的时候更新的幅度是比较大的，也就是说开始的学习率可以设置大一点，但是当训练集的 loss 下降到一定程度之后，，使用这个太大的学习率就会导致 loss 一直来回震荡，比如

![](https://ws4.sinaimg.cn/large/006tNc79ly1fmrvdlncomj30bf0aywet.jpg)

这个时候就需要对学习率进行衰减已达到 loss 的充分下降，而是用学习率衰减的办法能够解决这个矛盾，学习率衰减就是随着训练的进行不断的减小学习率。

在`tensorflow`中学习率衰减非常方便，使用 `tf.train.exponential_decay`, 但是它只支持指数式衰减和固定步长衰减, 不是很方便, 具体的可以参考[官方文档](https://tensorflow.google.cn/api_docs/python/tf/train/exponential_decay)

我们使用`placeholder`来实现学习率衰减会更加灵活

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
import tensorflow.contrib.slim as slim

from utils import cifar10_input
from utils import resnet

在这里, 我们用函数设计一个下降策略:
- 当训练步数小于12000时, 输出0.01
- 大于12000时输出0.001

In [ ]:
def lr_step(step, **kwargs):
    lr = tf.cond(tf.less(step, 12000), lambda: 0.1, lambda: 0.01)
    
    return lr

In [ ]:
# 导入数据
train_imgs, train_labels, val_imgs, val_labels = cifar10_input.load_data(data_dir='../经典卷积神经网络/cifar10_data', image_size=96)

In [ ]:
is_training = tf.placeholder(tf.bool, name='is_training')    

In [ ]:
with slim.arg_scope(resnet.resnet_arg_scope()):
    train_out = resnet.resnet(train_imgs, 10, is_training=is_training)
    val_out = resnet.resnet(val_imgs, 10, is_training=is_training, reuse=True)

In [ ]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

在这里我们用一个占位符来表示学习率, 方便在训练过程中我们从外部修改它

In [ ]:
train_step = tf.Variable(0, trainable=False, name='train_step')
lr = lr_step(train_step)

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss, global_step=train_step)

In [ ]:
from utils.learning import train_with_bn

In [ ]:
train_losses, train_accs, val_losses, val_accs = \
    train_with_bn(train_op, train_loss, train_acc, val_loss, val_acc, 20000, is_training=is_training, \
                 train_log_step=200, val_log_step=200)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
x_axis = np.arange(5, 101) * 200
plt.plot(x_axis, train_losses[5:], label='train')
plt.plot(x_axis, val_losses[5:], label='valid')
plt.xlabel('step')
plt.legend(loc='best')

这里我们训练了`20000`次，在`12000`次的时候进行了学习率衰减，可以看 loss 曲线在 20 次的时候不管是 train loss 还是 valid loss，都有了一个陡降。

当然这里我们只是作为举例，在实际应用中，做学习率衰减之前应该经过充分的训练，比如训练 80 个`epoch`或者 100 个`epoch`，然后再做学习率衰减得到更好的结果，有的时候甚至需要做多次学习率衰减